# Tool and Function Calling

This guide demonstrates how to use SGLang’s [Funcion calling](https://platform.openai.com/docs/guides/function-calling) functionality.

## OpenAI Compatible API

### Launching the Server

In [ ]:
from openai import OpenAI
import json
from sglang.utils import wait_for_server, print_highlight, terminate_process
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen2.5-14B-Instruct --tool-call-parser qwen25 --host 0.0.0.0"  # qwen25
)
wait_for_server(f"http://localhost:{port}")

INFO 03-04 21:55:16 __init__.py:190] Automatically detected platform cuda.
2025-03-04 21:55:18,186 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
[2025-03-04 21:55:21] server_args=ServerArgs(model_path='Qwen/Qwen2.5-14B-Instruct', tokenizer_path='Qwen/Qwen2.5-14B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-14B-Instruct', chat_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=36789, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=223558563, constrained_json_whitespace_pattern=None, watchdog

Note that `--tool-call-parser` defines the parser used to interpret responses. Currently supported parsers include:

- llama3: Llama 3.1 / 3.2 (e.g. meta-llama/Llama-3.1-8B-Instruct, meta-llama/Llama-3.2-1B-Instruct).
- mistral: Mistral (e.g. mistralai/Mistral-7B-Instruct-v0.3, mistralai/Mistral-Nemo-Instruct-2407, mistralai/
Mistral-Nemo-Instruct-2407, mistralai/Mistral-7B-v0.3).
- qwen25: Qwen 2.5 (e.g. Qwen/Qwen2.5-1.5B-Instruct, Qwen/Qwen2.5-7B-Instruct).

### Define Tools for Function Call
Below is a Python snippet that shows how to define a tool as a dictionary. The dictionary includes a tool name, a description, and property defined Parameters.

In [ ]:
# Define tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to find the weather for, e.g. 'San Francisco'",
                    },
                    "state": {
                        "type": "string",
                        "description": "the two-letter abbreviation for the state that the city is"
                        " in, e.g. 'CA' which would mean 'California'",
                    },
                    "unit": {
                        "type": "string",
                        "description": "The unit to fetch the temperature in",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["city", "state", "unit"],
            },
        },
    }
]

### Define Messages

In [ ]:
def get_messages():
    return [
        {
            "role": "user",
            "content": "What's the weather like in Boston today? Please think first and respond with the format: Today's weather is :{function call result}",
        }
    ]


messages = get_messages()

### Initialize the Client

In [ ]:
# Initialize OpenAI-like client
client = OpenAI(api_key="None", base_url=f"http://0.0.0.0:{port}/v1")
model_name = client.models.list().data[0].id

[2025-03-04 22:09:43] INFO:     127.0.0.1:46948 - "GET /v1/models HTTP/1.1" 200 OK


###  Non-Streaming Request

In [ ]:
# Non-streaming mode test
response_non_stream = client.chat.completions.create(
    model=model_name,
    messages=messages,
    temperature=0.8,
    top_p=0.8,
    stream=False,  # Non-streaming
    tools=tools,
)
print_highlight("Non-stream response:")
print(response_non_stream)
print_highlight("==== content ====")
print(response_non_stream.choices[0].message.content)
print_highlight("==== tool_calls ====")
print(response_non_stream.choices[0].message.tool_calls)

[2025-03-04 22:09:44 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 280, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-04 22:09:44 TP0] Decode batch. #running-req: 1, #token: 315, token usage: 0.00, gen throughput (token/s): 7.62, largest-len: 0, #queue-req: 0, 
[2025-03-04 22:09:44] INFO:     127.0.0.1:46948 - "POST /v1/chat/completions HTTP/1.1" 200 OK
Non-stream response:
ChatCompletion(id='c752e459485544ea80f9607fcb9687aa', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Today's weather is :", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='0', function=Function(arguments='{"city": "Boston", "state": "MA", "unit": "fahrenheit"}', name='get_current_weather'), type='function')], reasoning_content=None), matched_stop=151645)], created=1741097384, model='Qwen/Qwen2.5-14B-Instruct', object='chat.completion', service_tier=None, system_fingerpri

### Streaming Request

In [ ]:
# Streaming mode test
print_highlight("Streaming response:")
response_stream = client.chat.completions.create(
    model=model_name,
    messages=messages,
    temperature=0.8,
    top_p=0.8,
    stream=True,  # Enable streaming
    tools=tools,
)

chunks = []
for chunk in response_stream:
    chunks.append(chunk)
    if chunk.choices[0].delta.tool_calls:
        print(chunk.choices[0].delta.tool_calls[0])

Streaming response:
[2025-03-04 22:09:45] INFO:     127.0.0.1:46948 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-03-04 22:09:45 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 280, token usage: 0.00, #running-req: 0, #queue-req: 0, 
ChoiceDeltaToolCall(index=None, id='0', function=ChoiceDeltaToolCallFunction(arguments='', name='get_current_weather'), type='function')
ChoiceDeltaToolCall(index=None, id='0', function=ChoiceDeltaToolCallFunction(arguments='{"city": "', name=''), type='function')
ChoiceDeltaToolCall(index=None, id='0', function=ChoiceDeltaToolCallFunction(arguments='Boston"', name=''), type='function')
ChoiceDeltaToolCall(index=None, id='0', function=ChoiceDeltaToolCallFunction(arguments=', "state": "', name=''), type='function')
ChoiceDeltaToolCall(index=None, id='0', function=ChoiceDeltaToolCallFunction(arguments='MA"', name=''), type='function')
[2025-03-04 22:09:46 TP0] Decode batch. #running-req: 1, #token: 316, token usage: 0.00, gen throughput 


### Handle Tool Calls

When the engine determines it should call a particular tool, it will return arguments or partial arguments through the response. You can parse these arguments and later invoke the tool accordingly.

**Non-Streaming Request**

In [ ]:
name_non_stream = response_non_stream.choices[0].message.tool_calls[0].function.name
arguments_non_stream = (
    response_non_stream.choices[0].message.tool_calls[0].function.arguments
)

print_highlight(f"Final streamed function call name: {name_non_stream}")
print_highlight(f"Final streamed function call arguments: {arguments_non_stream}")

Final streamed function call name: get_current_weather
Final streamed function call arguments: {"city": "Boston", "state": "MA", "unit": "fahrenheit"}


**Streaming Request**

In [ ]:
# Parse and combine function call arguments
arguments = []
for chunk in chunks:
    choice = chunk.choices[0]
    delta = choice.delta
    if delta.tool_calls:
        tool_call = delta.tool_calls[0]
        if tool_call.function.name:
            print_highlight(f"Streamed function call name: {tool_call.function.name}")

        if tool_call.function.arguments:
            arguments.append(tool_call.function.arguments)
            print(f"Streamed function call arguments: {tool_call.function.arguments}")

# Combine all fragments into a single JSON string
full_arguments = "".join(arguments)
print_highlight(f"Final streamed function call arguments: {full_arguments}")

Streamed function call name: get_current_weather
Streamed function call arguments: {"city": "
Streamed function call arguments: Boston"
Streamed function call arguments: , "state": "
Streamed function call arguments: MA"
Streamed function call arguments: , "unit": "
Streamed function call arguments: f
Streamed function call arguments: ahrenheit"}
Final streamed function call arguments: {"city": "Boston", "state": "MA", "unit": "fahrenheit"}


### Define a Tool Function

In [ ]:
# This is a demonstration, define real function according to your usage.
def get_current_weather(city: str, state: str, unit: "str"):
    return (
        f"The weather in {city}, {state} is 85 degrees {unit}. It is "
        "partly cloudly, with highs in the 90's."
    )


available_tools = {"get_current_weather": get_current_weather}


## Execute the Tool

In [ ]:
call_data = json.loads(full_arguments)

messages.append(
    {
        "role": "user",
        "content": "",
        "tool_calls": {"name": "get_current_weather", "arguments": full_arguments},
    }
)

# Call the corresponding tool function
tool_name = messages[-1]["tool_calls"]["name"]
tool_to_call = available_tools[tool_name]
result = tool_to_call(**call_data)
print_highlight(f"Function call result: {result}")
messages.append({"role": "tool", "content": result, "name": tool_name})

print_highlight(f"Updated message history: {messages}")

Function call result: The weather in Boston, MA is 85 degrees fahrenheit. It is partly cloudly, with highs in the 90's.
Updated message history: [{'role': 'user', 'content': "What's the weather like in Boston today? Please think first and respond with the format: Today's weather is :{function call result}"}, {'role': 'user', 'content': '', 'tool_calls': {'name': 'get_current_weather', 'arguments': '{"city": "Boston", "state": "MA", "unit": "fahrenheit"}'}}, {'role': 'tool', 'content': "The weather in Boston, MA is 85 degrees fahrenheit. It is partly cloudly, with highs in the 90's.", 'name': 'get_current_weather'}]


### Send Results Back to Model

In [ ]:
final_response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    temperature=0.8,
    top_p=0.8,
    stream=False,
    tools=tools,
)
print_highlight("Non-stream response:")
print(final_response)

[2025-03-04 22:09:54 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 327, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-04 22:09:55] INFO:     127.0.0.1:46982 - "POST /v1/chat/completions HTTP/1.1" 200 OK
Non-stream response:
ChatCompletion(id='d2fdf68212aa46409f0d97be4af45222', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Today's weather is :85 degrees fahrenheit, partly cloudy with highs in the 90's.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, reasoning_content=None), matched_stop=151645)], created=1741097395, model='Qwen/Qwen2.5-14B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=328, total_tokens=351, completion_tokens_details=None, prompt_tokens_details=None))


## Native API and SGLang Runtime (SRT)

In [ ]:
from transformers import AutoTokenizer
import requests

# generate an answer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-14B-Instruct")

messages = get_messages()

input = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    tools=tools,
)

gen_url = f"http://localhost:{port}/generate"
gen_data = {"text": input, "sampling_params": {"skip_special_tokens": False}}
gen_response = requests.post(gen_url, json=gen_data).json()["text"]
print_highlight("Reponse")
print(gen_response)

# parse the response
parse_url = f"http://localhost:{port}/parse_function_call"

function_call_input = {
    "text": gen_response,
    "tool_call_parser": "qwen25",
    "tools": tools,
}

function_call_response = requests.post(parse_url, json=function_call_input)
function_call_response_json = function_call_response.json()
print_highlight("==== Calls ====")
print("function name: ", function_call_response_json["calls"][0]["name"])
print("function arguments: ", function_call_response_json["calls"][0]["parameters"])

ValueError: Tools should either be a JSON schema, or a callable function with type hints and a docstring suitable for auto-conversion to a schema.

In [ ]:
terminate_process(server_process)

## Offline Engine API

In [ ]:
import sglang as sgl
from sglang.srt.function_call_parser import FunctionCallParser
from sglang.srt.managers.io_struct import Tool, Function

llm = sgl.Engine(model_path="Qwen/Qwen2.5-14B-Instruct")
tokenizer = llm.tokenizer_manager.tokenizer
input_ids = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, tools=tools
)

sampling_params = {
    "max_new_tokens": 128,
    "temperature": 0.3,
    "top_p": 0.95,
    "skip_special_tokens": False,
}

# 1) Offline generation
result = llm.generate(input_ids=input_ids, sampling_params=sampling_params)
generated_text = result["text"]  # Assume there is only one prompt

print("=== Offline Engine Output Text ===")
print(generated_text)


# 2) Parse using FunctionCallParser
def convert_dict_to_tool(tool_dict: dict) -> Tool:
    function_dict = tool_dict.get("function", {})
    return Tool(
        type=tool_dict.get("type", "function"),
        function=Function(
            name=function_dict.get("name"),
            description=function_dict.get("description"),
            parameters=function_dict.get("parameters"),
        ),
    )


tools = [convert_dict_to_tool(raw_tool) for raw_tool in tools]

parser = FunctionCallParser(tools=tools, tool_call_parser="qwen25")
normal_text, calls = parser.parse_non_stream(generated_text)

print("\n=== Parsing Result ===")
print("Normal text portion:", normal_text)
print("Function call portion:")
for call in calls:
    # call: ToolCallItem
    print(f"  - tool name: {call.name}")
    print(f"    parameters: {call.parameters}")

# 3) If needed, perform additional logic on the parsed functions, such as automatically calling the corresponding function to obtain a return value, etc.

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/usr/local/conda/lib/python3.9/multiprocessing/resource_tracker.py:96: UserWarning: resource_tracker: process died unexpectedly, relaunching.  Some resources might leak.
  warnings.warn('resource_tracker: process died unexpectedly, '
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parall

INFO 03-04 22:16:26 __init__.py:190] Automatically detected platform cuda.
INFO 03-04 22:16:26 __init__.py:190] Automatically detected platform cuda.


2025-03-04 22:16:27,731 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  12% Completed | 1/8 [00:00<00:02,  2.81it/s]
Loading safetensors checkpoint shards:  25% Completed | 2/8 [00:00<00:02,  2.58it/s]
Loading safetensors checkpoint shards:  38% Completed | 3/8 [00:01<00:01,  2.58it/s]
Loading safetensors checkpoint shards:  50% Completed | 4/8 [00:01<00:01,  2.58it/s]
Loading safetensors checkpoint shards:  62% Completed | 5/8 [00:01<00:01,  2.58it/s]
Loading safetensors checkpoint shards:  75% Completed | 6/8 [00:02<00:00,  2.59it/s]
Loading safetensors checkpoint shards:  88% Completed | 7/8 [00:02<00:00,  2.62it/s]
Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:02<00:00,  3.11it/s]
Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:02<00:00,  2.78it/s]

2025-03-04 22:16:34,933 - INFO - flashinfer.jit: Loading JIT o

=== Offline Engine Output Text ===
Today's weather is :<tool_call>
{"name": "get_current_weather", "arguments": {"city": "Boston", "state": "MA", "unit": "fahrenheit"}}
</tool_call>

=== Parsing Result ===
Normal text portion: Today's weather is :
Function call portion:
  - tool name: get_current_weather
    parameters: {"city": "Boston", "state": "MA", "unit": "fahrenheit"}


In [ ]:
llm.shutdown()

## How to support a new model?
1. Update the TOOLS_TAG_LIST in sglang/srt/function_call_parser.py with the model’s tool tags. Currently supported tags include:
```
	TOOLS_TAG_LIST = [
	    “<|plugin|>“,
	    “<function=“,
	    “<tool_call>“,
	    “<|python_tag|>“,
	    “[TOOL_CALLS]”
	]
```
2. Create a new detector class in sglang/srt/function_call_parser.py that inherits from BaseFormatDetector. The detector should handle the model’s specific function call format. For example:
```
    class NewModelDetector(BaseFormatDetector):
```
3. Add the new detector to the MultiFormatParser class that manages all the format detectors.